In [104]:
import datasets
import random
lines = list(open("sentences.txt", "r").readlines())

def generate_filler():
    if random.random() < 0.2:
        return "like"
    else:
        uh = "u"
        use_h = bool(random.getrandbits(1))
        original_h = use_h
        while use_h and len(uh) < 5:
            uh += "h"
            use_h = bool(random.getrandbits(1))

        while not original_h or (bool(random.getrandbits(1)) and len(uh) < 5):
            uh += "m"
            original_h = True
        
        return uh

def generate_stammer(word):
    index_to_repeat = random.randrange(1, min(3, len(word))) if len(word) > 1 else 1
    times_to_repeat = random.randrange(1, 3)
    return (word[:index_to_repeat] + "-")*times_to_repeat

def augment(line):
    words = line.lower().split(" ")
    filler_num = random.randrange(0, 5)
    stammer_num = random.randrange(0, 6-filler_num)
    repeat_num = random.randrange(0, 3)
    labels = [0]*len(words)

    for _ in range(filler_num):
        idx = random.randrange(0, len(words))
        words.insert(idx, generate_filler())
        labels.insert(idx, 1)

    for _ in range(stammer_num):
        idx = random.randrange(0, len(words))
        words.insert(idx, generate_stammer(words[idx]))
        labels.insert(idx, 1)

    for _ in range(repeat_num):
        idx = random.randrange(0, len(words))
        words.insert(idx, words[idx])
        labels.insert(idx, 1)

    return words, labels

new_ds = {"tokens": [], "labels": []}
for _ in range(3):
    for line in lines:
        if line:
            w, l = augment(line.strip())
            new_ds["tokens"].append(w)
            new_ds["labels"].append(l)

In [106]:
import datasets
ds = datasets.Dataset.from_dict(new_ds)
ds.save_to_disk("./stutter-ds-2")

Saving the dataset (0/1 shards):   0%|          | 0/2172 [00:00<?, ? examples/s]